In [10]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import base64
import io
import psycopg2
from moviepy.editor import VideoFileClip

In [12]:
app = dash.Dash()

app.layout = html.Div([
    # Title
    html.Div(
        children=[
            html.H1('Video Analysis',
                    style={'textAlign': 'center'}),
        ]),

    # Upload Video
    html.Div(
        dcc.Upload(
            id='upload-video',
            children=html.Div([
                'Drag and Drop or Select a Video',
            ]),
            style={
                'width': '100%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            # Allow multiple files to be uploaded        
            multiple=True,
        ),
        style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center'}
    ),

    # Output Video
    html.Div(id='output-video',
             style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center'})
])

@app.callback(
    Output('output-video', 'children'),
    Input('upload-video', 'contents'),
    prevent_initial_call=True
)
def update_output(list_of_contents):
    if list_of_contents is not None:

        # Connect to your postgres DB
        conn = psycopg2.connect("dbname=VideoWeb user=postgres password=1q2w3e")

        # Open a cursor to perform database operations
        cur = conn.cursor()

        # Create table if it doesn't exist
        cur.execute("CREATE TABLE IF NOT EXISTS video_metadata (id serial PRIMARY KEY, duration REAL, fps REAL, size TEXT, audio_fps REAL);")

        for contents in list_of_contents:
            # The contents are base64 encoded, so we must decode them
            decoded = base64.b64decode(contents.split(',')[1])

            # Write the binary data to a file
            with open('temp.mp4', 'wb') as f:
                f.write(decoded)

            # Read the video file and extract metadata
            clip = VideoFileClip('temp.mp4')
            duration = clip.duration
            fps = clip.fps
            size = str(clip.size)
            audio_fps = clip.audio.fps if clip.audio is not None else None

            # Insert the metadata into the database
            cur.execute("INSERT INTO video_metadata (duration, fps, size, audio_fps) VALUES (%s, %s, %s, %s);", (duration, fps, size, audio_fps))

        # Commit changes and close resources
        conn.commit()
        cur.close()
        conn.close()       

        children = [
            html.Video(src=contents, 
                       controls=True, 
                       style={'width': '50%'}) for contents in list_of_contents
        ]
        return children

if __name__ == '__main__':
    app.run_server(debug=True)